In [67]:
# 환경변수
from dotenv import load_dotenv
load_dotenv()
import os 
import sys
# 라이브러리
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [68]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [69]:
loader = PyMuPDFLoader("data/valorant_tutorial.pdf")

docs = loader.load()

text_spliteer = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_spliteer.split_documents(docs)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [78]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-preview-04-17', temperature=0.1)
prompt = PromptTemplate.from_template(
"""
역할 : 
당신은 발로란트 게임에 대해 알려주는 친절한 인게임 AI 어시스턴트입니다. 주 대상은 발로란트 게임을 처음 접하는 사람들입니다.
질문에 대하여 간결하고 명확한 답변을 제공하세요.
답변을 먼저 말하고, 문서 검색 결과에 대해서는 언급하지 마세요.
분량은 2-3문장 정도로 작성하세요.
당신은 발로란트 게임의 공식 문서와 관련된 정보를 바탕으로 답변을 작성합니다.
모르는 내용은 "확인할 수 없습니다."라고 답변하세요.
---------------------------------------------------------------
다음 질문에 답변하세요: 
{question}\n\n
---------------------------------------------------------------
아래는 관련된 정보입니다:
\n{context}\n\n
답변:
"""
)
chain ={"context":retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
chain.invoke("제트의 특징은 무엇인가요?")

'안녕하세요! 발로란트 인게임 AI 어시스턴트입니다.\n\n제트는 빠른 기동성을 활용하여 공격적인 플레이를 펼치는 요원입니다. 대시와 상승 기류 스킬을 사용해 빠르게 위치를 변경하거나 높은 곳으로 이동하며 적을 기습하는 데 특화되어 있습니다.'

In [71]:
router_llm = llm

router_prompt = PromptTemplate.from_template("""
        당신은 발로란트 게임 내에서 유저의 질문 유형을 분류하는 AI입니다. 다음 질문을 분석하여 해당 질문의 유형을 결정하세요. 오직 분류 결과만 출력하세요. \n
        질문: {question}\n\n질문 유형을 다음 중 하나로 분류하세요: 'game_tutorial', 'agent_infomation', 'unclear'
        """)


router_chain = router_prompt | router_llm | StrOutputParser()

question_type = router_chain.invoke({"question": "내가 지금 뭐해야 하지?"})

In [72]:
question_type

'unclear'

In [96]:
from rag_bot import tutorial_chain
tutorial_chain.invoke( "지금 첫 라운드인데 어떻게 해야 할까요?")

'첫 라운드에는 크레딧이 제한적이니, 권총과 기본 스킬을 구매하는 것이 좋아요. 팀원들과 함께 어디로 갈지 정하고, 서로 협력하며 목표를 달성해 보세요!'

In [1]:
from main import graph_app


c:\Users\user\miniconda3\envs\graph_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
answer = graph_app.invoke({"question": "조작법을 모르겠어"})['answer']

---라우터 노드 실행---
질문 유형: game_tutorial
---게임 튜토리얼 질문 처리---
간단한 답변 품질 평가
답변 평가 내역: 적절


In [41]:
answer

'조작법은 게임 내 설정 메뉴에서 확인할 수 있습니다. Esc 키를 눌러 설정으로 이동한 후, 조작법 탭에서 모든 키 설정을 보고 변경할 수 있습니다.'

In [102]:
from main import MyState

state = MyState()
type(state)

dict

In [8]:
answer

'발로란트에서는 Shift 키로 걸어서 발소리를 줄일 수 있고, Ctrl 키로 웅크릴 수 있습니다. 팀원과의 소통을 위해 Caps Lock 키로 지도를 열거나 Z 키로 핑을 찍을 수 있습니다. 또한 G 키로 무기를 버리거나 B 키로 구매 메뉴를 열 수 있습니다.'

In [44]:
from openai import OpenAI

client = OpenAI()
speech_file_path = "./answer_speech.wav"

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="nova",
    input=answer,
    instructions="Say clearly and say it in a calm yet majestic tone, like the 'Welcome to Summoner's Rift' line from League of Legends.",
    speed=1.0,
) as response:
    response.stream_to_file(speech_file_path)